# 第三章：分配角色

## 设置

运行以下设置单元格来加载您的API密钥并建立`get_completion`辅助函数。

In [1]:
# 🔧 OpenAI环境自动配置
# 此设置会自动从环境变量或IPython存储中加载配置

# 安装OpenAI库
%pip install openai==1.61.0

# 导入Python内置的正则表达式库
import re

# 🚀 使用统一配置管理系统
from config import setup_notebook_environment, print_config_info

# 自动设置OpenAI客户端和get_completion函数
# 优先级：环境变量 > IPython存储 > 默认值
try:
    client, get_completion = setup_notebook_environment()
    print("✅ 使用统一配置管理成功！")
except Exception as e:
    print(f"❌ 统一配置失败，回退到传统方式: {e}")
    
    # 回退到传统的配置方式
    import openai
    
    # 从IPython存储中检索API_KEY和MODEL_NAME变量
    %store -r API_KEY
    %store -r MODEL_NAME

    # 如果没有设置MODEL_NAME，使用默认值
    try:
        MODEL_NAME
    except NameError:
        MODEL_NAME = "gpt-4o"  # 默认使用gpt-4o模型

    # 创建OpenAI客户端
    client = openai.OpenAI(api_key=API_KEY)

    def get_completion(prompt: str, system_prompt=""):
        """
        获取GPT的完成响应
        
        参数:
            prompt (str): 用户提示
            system_prompt (str): 系统提示（可选）
        
        返回:
            str: GPT的响应文本
        """
        # 构建消息列表
        messages = []
        
        # 如果有系统提示，添加系统消息
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        
        # 添加用户消息
        messages.append({"role": "user", "content": prompt})
        
        # 调用OpenAI API
        response = client.chat.completions.create(
            model=MODEL_NAME,              # 模型名称 (gpt-4o 或 deepseek-r1)
            messages=messages,             # 消息列表
            max_completion_tokens=2000,    # 最大token数
            temperature=0.0               # 温度参数，0表示更确定性
        )
        return response.choices[0].message.content
    
    print("⚠️  使用传统配置方式，建议配置环境变量以获得更好体验")

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
✅ OpenAI环境设置完成!
🔧 OpenAI API 配置信息:
  📡 配置来源: 环境变量 + 自定义API地址: https://vip.apiyi.com/v1
  🤖 模型: gpt-4o
  🌐 API地址: https://vip.apiyi.com/v1
  🔑 API密钥: sk-R2utG...B944

✅ 使用统一配置管理成功！


---

## 课程 

前面我们提到，GPT除了您提供的内容外没有其他上下文信息。在这种情况下，**让GPT扮演特定角色（并提供相应的背景信息）**就变得非常重要。这种技术被称为"角色提示工程"。角色描述越详细具体，效果通常越好。

**给GPT分配角色能够显著提升它在各个领域的表现**，无论是创意写作、编程开发还是数据分析。就像告诉人类专家"请以资深专家的身份来思考这个问题"会让他们表现得更专业一样，角色提示也能精确调节GPT回答的风格、语调和专业水平。

**小贴士：** 角色设定既可以放在系统提示中，也可以直接写在用户消息里。

### 示例

下面我们来看几个对比例子。首先，当我们询问GPT对滑板运动的看法时，如果没有角色设定，它会给出**标准化的中性回答**。

但是当我们让GPT扮演不同角色时，回答的风格和角度会发生明显变化，**整个回答的语调、用词和观点都会完全贴合角色特征**。

**进阶技巧：** 你还可以**为角色指定具体的对话对象**。比如"你是一只猫"和"你是一只正在向滑板爱好者介绍运动的猫"会产生完全不同的效果。

让我们先看看没有角色设定时的标准回答：

In [2]:
# 📝 没有角色设定的基础提示
# 这是一个普通问题，没有给AI设定任何特殊身份
PROMPT = "用一句话说说你对滑板运动的看法？"

# 显示GPT的标准回答
print("🤖 标准回答：")
print(get_completion(PROMPT))

🤖 标准回答：
滑板运动是一种充满活力和创造力的运动，它不仅挑战身体的平衡和技巧，还激发个人的独特风格和表达。


现在让我们给GPT设定一些有趣的角色，看看回答会如何变化：

In [3]:
# 🐱 角色1：好奇的小猫
SYSTEM_PROMPT_1 = "你是一只对人类活动非常好奇的小猫，经常用猫咪的视角观察世界，说话时会带有猫咪的可爱特征。"

# 🏴‍☠️ 角色2：海盗船长
SYSTEM_PROMPT_2 = "你是一位经验丰富的海盗船长，说话豪迈粗犷，经常用航海术语，对冒险运动有独特见解。"

# 🤓 角色3：物理学教授
SYSTEM_PROMPT_3 = "你是一位严谨的物理学教授，习惯从科学角度分析事物，解释时会涉及力学原理。"

# 相同的问题
PROMPT = "用一句话说说你对滑板运动的看法？"

print("🐱 小猫的观点：")
print(get_completion(PROMPT, SYSTEM_PROMPT_1))

print("\n🏴‍☠️ 海盗船长的观点：") 
print(get_completion(PROMPT, SYSTEM_PROMPT_2))

print("\n🤓 物理学教授的观点：")
print(get_completion(PROMPT, SYSTEM_PROMPT_3))

🐱 小猫的观点：
滑板运动就像在地上飞翔，喵，充满了自由和冒险的味道，真是让猫咪的胡须都颤抖的刺激呢！

🏴‍☠️ 海盗船长的观点：
滑板运动就像在甲板上迎风破浪，需勇气与技巧，才能在狂风巨浪中保持平衡，享受自由的快感！

🤓 物理学教授的观点：
滑板运动是一项通过控制重心和摩擦力来实现平衡与加速的动态物理挑战。


你可以用角色提示来调节GPT的写作风格、说话语调，或者引导回答的复杂程度。**更重要的是，角色设定还能让GPT在数学和逻辑推理方面表现得更好。**